In [20]:
!pip install ibm_watson bs4

In [21]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [26]:
recognition_service = SpeechToTextV1(IAMAuthenticator('xI_tN7DsHSXeQs5MN8bO14lBB0wsI_26Y5QiPd8Y-C-g'))
recognition_service.set_service_url('https://api.eu-de.speech-to-text.watson.cloud.ibm.com/instances/1e2f82e2-33f2-4531-830b-3ffff1fc1aa8')
SPEECH_EXTENSION = "*.ogg"
SPEECH_AUDIOTYPE = "audio/ogg"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

In [27]:
assistant = AssistantV2(version='2020-09-24', authenticator=IAMAuthenticator('WOlqT-9bohtKEfoBjPtwlde7syItJQYi0WMcNSUi1588'))
assistant.set_service_url('https://api.eu-de.assistant.watson.cloud.ibm.com/instances/94e0fd04-bac7-40e1-a572-23f9d9938e20')
ASSISTANT_ID = "881646cf-d2b5-4fd2-b392-b90d934b2844"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

In [28]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('n-PukXQ8i_MK-j3oVI4Qar9iM7oJW0GA20U43-jETn2O'))
synthesis_service.set_service_url('https://api.eu-de.text-to-speech.watson.cloud.ibm.com/instances/5786e645-47de-4543-baf5-1fcf5d5f740a')

def speak_text(text):
    with open('temp.ogg', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/ogg', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.ogg", autoplay=True)

In [29]:
speak_text(message_assistant(recognize_audio()))